In [1]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import OpenAI
from dotenv import load_dotenv
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.agents import tool
import os
import json
import re
from langchain import Anthropic
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage,
)
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.agents import tool
from pydantic import BaseModel, Field
from solcx import install_solc, compile_source
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from utils.tools import compile_solidity, SolidityCompilerInput


load_dotenv()
import json

def get_secret(name):
    if name in os.environ:
        return os.environ[name]
    return os.getenv(name)

In [2]:
@tool(return_direct=False)
def compile_solidity(solidity_code_str):
    """
    This function compiles Solidity files using a specific version of solc.
    :return: Dict, the compiled contracts.
    """
    solidity_code_str = solidity_code_str.strip('`')
    solc_version = '0.8.2'
    # Install the specified version of solc
    install_solc(solc_version)
    try:
        result = compile_source(solidity_code_str)
    except Exception as e:
        print(f"couldn't compile {solidity_code_str}\n{str(e)}")
        return f'Compilation Failed. Either the code is invalid or current compiler({solc_version}) didnt match'
    print(result)
    return 'Compilation Successful. It is a valid solidity code.'

class SolidityCompilerInput(BaseModel):
    question: str = Field(description="Should be pure solidity code without surrounded backticks")

In [3]:
# !solc-select install 0.8.9
!solc-select use 0.8.9

Switched global version to 0.8.9


In [4]:
solidity_function = """// SPDX-License-Identifier: MIT
pragma solidity ^0.8.9;

contract Gas {
    uint public i = 0;

    // Using up all of the gas that you send causes your transaction to fail.
    // State changes are undone.
    // Gas spent are not refunded.
    function forever() public {
        // Here we run a loop until all of the gas are spent
        // and the transaction fails
        while (true) {
            i += 1;
        }
    }
}
"""

## Initialize an agent with 1 tool

In [5]:
llm = OpenAI(temperature=0)
# llm = ChatAnthropic(model="claude-v1-100k", temperature=0., max_tokens_to_sample=4096)
tools = [
    Tool(
        func=compile_solidity,
        name="solidity compiler",
        description="Can compile solidity code. Accepts ONLY solidity code",
        args_schema=SolidityCompilerInput,
    ),
]
self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
self_ask_with_search.run(f"is this a valid solidity code:\n{solidity_function}")



> Entering new AgentExecutor chain...
 I need to check if this is valid solidity code
Action: solidity compiler
Action Input: // SPDX-License-Identifier: MIT
pragma solidity ^0.8.9;

contract Gas {
    uint public i = 0;

    // Using up all of the gas that you send causes your transaction to fail.
    // State changes are undone.
    // Gas spent are not refunded.
    function forever() public {
        // Here we run a loop until all of the gas are spent
        // and the transaction fails
        while (true) {
            i += 1;
        }
    }
}
{'<stdin>:Gas': {'abi': [{'inputs': [], 'name': 'forever', 'outputs': [], 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [], 'name': 'i', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'stateMutability': 'view', 'type': 'function'}], 'asm': {'.code': [{'begin': 57, 'end': 432, 'name': 'PUSH', 'source': 0, 'value': '80'}, {'begin': 57, 'end': 432, 'name': 'PUSH', 'source': 0, 'value': '40'}, {

Thought: I now know the final answer
Final Answer: Yes, this is a valid solidity code.

> Finished chain.


'Yes, this is a valid solidity code.'